In [1]:
LIGHT_IMPORT = True

%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity checks
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [7]:
df.columns.tolist()

['user_id',
 'age',
 'status',
 'sex',
 'orientation',
 'body_type',
 'diet',
 'drinks',
 'drugs',
 'education',
 'ethnicity',
 'height',
 'income',
 'job',
 'last_online',
 'location',
 'offspring',
 'pets',
 'religion',
 'sign',
 'smokes',
 'speaks',
 'essay0',
 'essay1',
 'essay2',
 'essay3',
 'essay4',
 'essay5',
 'essay6',
 'essay7',
 'essay8',
 'essay9',
 'bio_text',
 'lat',
 'lon',
 'diet_c',
 'drinks_c',
 'smokes_c',
 'drugs_c']

In [3]:
df.head(5)

,user_id,age,status,sex,orientation,body_type,diet,drinks,drugs,education,...,essay7,essay8,essay9,bio_text,lat,lon,diet_c,drinks_c,smokes_c,drugs_c
0,0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,...,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,37.653540,-122.416866,omnivore,light,light,none
1,1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,...,,i am very open and will share just about anyth...,,i am a chef: this is what that means. 1. i am ...,37.804456,-122.271356,other,heavy,none,sometimes
2,2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,...,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...","i'm not ashamed of much, but writing public te...",37.779259,-122.419329,omnivore,light,none,
3,3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,...,,,you feel so inclined.,i work in a library and go to school. . . <e> ...,37.870839,-122.272863,vegetarian,light,none,
4,4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,...,,,,hey how's it going? currently vague on the pro...,37.779259,-122.419329,,light,none,none


In [14]:
import pandas as pd

# Take only the first 10 characters (YYYY-MM-DD)
df["last_online_clean"] = df["last_online"].astype(str).str.slice(0, 10)

# Parse as date
df["last_online_dt"] = pd.to_datetime(df["last_online_clean"], errors="coerce", utc=True)

# Now build recency features
df["recency_rank"] = df["last_online_dt"].rank(ascending=True)        # oldest=1
mask = df["last_online_dt"].notna()

# Normalized 0–1 scale within dataset
min_dt, max_dt = df["last_online_dt"].min(), df["last_online_dt"].max()
if pd.notna(min_dt) and pd.notna(max_dt) and max_dt > min_dt:
    span = (max_dt - min_dt).total_seconds()
    df["recency_norm"] = ((df["last_online_dt"] - min_dt).dt.total_seconds() / span).fillna(0.0)
else:
    df["recency_norm"] = 0.0

print(df[["last_online_dt","recency_norm"]].head(10))

             last_online_dt  recency_norm
0 2012-06-28 00:00:00+00:00      0.991892
1 2012-06-29 00:00:00+00:00      0.994595
2 2012-06-27 00:00:00+00:00      0.989189
3 2012-06-28 00:00:00+00:00      0.991892
4 2012-06-27 00:00:00+00:00      0.989189
5 2012-06-29 00:00:00+00:00      0.994595
6 2012-06-25 00:00:00+00:00      0.983784
7 2012-06-29 00:00:00+00:00      0.994595
8 2012-06-29 00:00:00+00:00      0.994595
9 2012-06-28 00:00:00+00:00      0.991892


In [25]:
def _ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    c = cands.copy()
    if "text_sim" not in c.columns and "score" in c.columns:
        c["text_sim"] = c["score"]
    c["text_sim"] = c["text_sim"].fillna(0).clip(lower=0)
    return c

In [118]:
def tinder_score(user_id, k=20, pool_k=500, max_km=50,drinks_strict=False, smokes_strict=False, 
                 drugs_strict=False,allow_missing=True,weights=None):
    """
    Tinder-style scorer:
      - Hard filters: orientation, location (<= max_km), age
      - Soft features: text_sim (from semantic score), lifestyle_score, age_score, loc_score
      - Weighted blend -> final_score
    """
    if weights is None:
        weights = {"text":0.70, "age":0.10, "life":0.15, "loc":0.05}

    # 1) Candidate pool (no hard lifestyle gates)
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
        use_age=True,use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
        drinks_strict=drinks_strict, smokes_strict=smokes_strict, drugs_strict=drugs_strict,
        allow_missing=True).copy()

    # 2) Component scores
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,
        drinks_strict=drinks_strict, smokes_strict=smokes_strict, drugs_strict=drugs_strict,
        allow_missing=allow_missing)
    cands = _ensure_text_sim(cands)  # maps 'score' -> 'text_sim' if present

    # 3) Blend
    ranked = apply_weights_tinder(cands, w_text=weights["text"], w_age=weights["age"],w_life=weights["life"], 
                           w_loc=weights["loc"]).copy()

    # 4) Final shape
    want = ["user_id","tinder_score","text_sim","age_score","lifestyle_score","loc_score","distance_km",
            "location","age","sex","orientation"]
    for col in want:
        if col not in ranked.columns:
            ranked[col] = np.nan
    ranked_top = ranked[want].head(k).copy()
    ranked_top["source"] = "ranked"

    # align random suggestions to same columns
    for col in want:
        if col not in random_potential_matches.columns:
            random_potential_matches[col] = np.nan
    return ranked[want].head(k).reset_index(drop=True)

In [119]:
tinder = tinder_score(user_id=14, k=10, pool_k=500, max_km=50)
tinder.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225


,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,38159,0.836038,0.742732,1.0,1.0,1.0,26.175450,"san carlos, california",32,m,straight
1,3717,0.830340,0.734164,1.0,1.0,1.0,31.203141,"palo alto, california",37,m,straight
2,15046,0.827298,0.728997,1.0,1.0,1.0,23.920071,"san francisco, california",36,m,straight
3,16398,0.825377,0.725979,1.0,1.0,1.0,31.203141,"palo alto, california",35,m,straight
4,35672,0.825033,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight
5,10444,0.823391,0.722858,1.0,1.0,1.0,19.197039,"berkeley, california",22,m,straight
6,47397,0.823200,0.722558,1.0,1.0,1.0,23.920071,"san francisco, california",32,m,straight
7,31015,0.823117,0.723199,1.0,1.0,1.0,23.920071,"san francisco, california",31,m,straight
8,41489,0.822003,0.720677,1.0,1.0,1.0,31.716076,"belvedere tiburon, california",23,m,straight
9,17586,0.820568,0.718421,1.0,1.0,1.0,23.920071,"san francisco, california",24,m,straight


In [120]:
# Merge recency_norm, blend, and nudge active users

def apply_weights_tinder(cands,w_text=0.60, w_age=0.10, w_life=0.10, w_loc=0.10, w_rec=0.10):
    """
    tinder_score = w_text*text_sim + w_age*age_score + w_life*lifestyle_score + w_loc*loc_score + w_rec*recency_norm
    (weights are normalized inside)
    """
    c = cands.copy()
    # ensure required cols exist
    for col in ["text_sim","age_score","lifestyle_score","loc_score"]:
        if col not in c.columns:
            c[col] = 0.0

    # bring in recency_norm from the master df
    c = c.merge(df[["user_id","recency_norm"]], on="user_id", how="left")
    c["recency_norm"] = c["recency_norm"].fillna(0.0).clip(0,1)

    # normalize weights
    total = float(w_text + w_age + w_life + w_loc + w_rec)
    w_text, w_age, w_life, w_loc, w_rec = [w/total for w in (w_text, w_age, w_life, w_loc, w_rec)]

    c["tinder_score"] = (w_text*c["text_sim"] + w_age *c["age_score"] + w_life*c["lifestyle_score"] +
                        w_loc *c["loc_score"] + w_rec *c["recency_norm"])
    return c.sort_values("tinder_score", ascending=False)


In [121]:
scored = compute_component_scores(user_id=u, cands=cands, max_km=20,
    drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True)
scored = _ensure_text_sim(scored)
scored.head(15)

,user_id,age,sex,orientation,location,text_sim,diet_c,drinks_c,smokes_c,drugs_c,lat,lon,age_score,lifestyle_score,loc_score,distance_km,dist_penalty
2,38159,32,m,straight,"san carlos, california",0.742732,,light,none,none,37.504936,-122.261823,1.0,1.00,0.691228,26.175450,6.175450
7,40022,25,m,straight,"san francisco, california",0.736307,omnivore,heavy,light,,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
9,25427,29,m,straight,"mountain view, california",0.735156,vegetarian,light,none,none,37.389389,-122.083210,1.0,0.75,0.107034,37.859321,17.859321
10,3717,37,m,straight,"palo alto, california",0.734164,,,light,,37.444329,-122.159847,1.0,1.00,0.439843,31.203141,11.203141
11,28563,29,m,straight,"san francisco, california",0.732910,,light,none,none,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
12,39450,25,m,straight,"san francisco, california",0.732575,omnivore,,none,none,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
16,23663,27,m,straight,"san mateo, california",0.730931,omnivore,light,none,none,37.496904,-122.333057,1.0,1.00,0.511777,29.764458,9.764458
20,15046,36,m,straight,"san francisco, california",0.728997,omnivore,light,none,none,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
21,33656,26,m,straight,"san francisco, california",0.728639,omnivore,light,none,none,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
26,46695,31,m,straight,"palo alto, california",0.726020,vegetarian,light,none,none,37.444329,-122.159847,1.0,0.75,0.439843,31.203141,11.203141


In [73]:
ranked = apply_weights_tinder(scored, w_text=0.55, w_age=0.10, w_life=0.10, w_loc=0.10, w_rec=0.15)
ranked[["user_id","tinder_score","text_sim","age_score","lifestyle_score","loc_score","recency_norm","distance_km"]].head(10)

,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,recency_norm,distance_km
8,44908,0.799900,0.636920,1.0,1.0,1.000000,0.997297,13.980956
11,4120,0.797826,0.633149,1.0,1.0,1.000000,0.997297,13.980956
16,57214,0.794873,0.627041,1.0,1.0,1.000000,1.000000,13.980956
27,56064,0.791719,0.621308,1.0,1.0,1.000000,1.000000,13.980956
2,47209,0.790329,0.644579,1.0,1.0,1.000000,0.905405,13.980956
26,7169,0.790105,0.621321,1.0,1.0,1.000000,0.989189,18.918516
15,26536,0.789406,0.627873,1.0,1.0,0.944814,0.997297,21.103721
19,30403,0.788710,0.624682,1.0,1.0,1.000000,0.967568,13.980956
1,43841,0.787977,0.654167,1.0,1.0,0.785908,0.997297,24.281842
35,53269,0.787719,0.615508,1.0,1.0,1.000000,0.994595,13.980956


In [125]:
def tinder_suggestions(user_id, random_sample=10, pool_k=500, max_km=50,drinks_strict=False, 
                       smokes_strict=False, drugs_strict=False,allow_missing=True, weights=None):
    """
    Returns a small set of 'suggested' candidates sampled from the same Tinder pool,
    scored the same way as ranked, and weighted with the same weights.
    - Does NOT return ranked; call tinder_score(...) separately.
    - Use exclude_ids to avoid duplicates with ranked results.
    """
    if weights is None:
        # same defaults you used for tinder_score
        weights = {"text": 0.70, "age": 0.10, "life": 0.15, "loc": 0.05}

    # 1) Pool with only hard gates (orientation, age, location)
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
        use_age=True,use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
        drinks_strict=drinks_strict, smokes_strict=smokes_strict, drugs_strict=drugs_strict,
        allow_missing=allow_missing).copy()

    # 2) Component scores (same as ranked path)
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,drinks_strict=drinks_strict, 
        smokes_strict=smokes_strict, drugs_strict=drugs_strict,allow_missing=allow_missing)
    cands = _ensure_text_sim(cands)

    # 3) Sample a few suggestions from the *scored* pool
    WANT_COLS = ["user_id","text_sim","age_score","lifestyle_score","loc_score","distance_km","location",
                 "age","sex","orientation"]
    if not set(WANT_COLS).issubset(cands.columns):
        for col in WANT_COLS:
            if col not in cands.columns:
                cands[col] = np.nan

    sugg = cands.sample(n=min(random_sample, len(cands)), random_state=42)[WANT_COLS].copy()

    # 4) Weight them exactly like ranked
    def _blend(df_in):
        return apply_weights_tinder(df_in,w_text=weights.get("text", 0.65),w_age =weights.get("age",  0.10),
                w_life=weights.get("life", 0.15),w_loc =weights.get("loc",  0.05),w_rec =weights.get("rec",  0.05))

    sugg_ranked = _blend(sugg).copy()
    sugg_ranked["source"] = "suggested"

    WANT_COLS_PLUS = ["user_id","tinder_score","text_sim","age_score","lifestyle_score",
                      "loc_score","distance_km","location","age","sex","orientation","source"]
    for col in WANT_COLS_PLUS:
        if col not in sugg_ranked.columns:
            sugg_ranked[col] = np.nan

    return sugg_ranked[WANT_COLS_PLUS].reset_index(drop=True)

In [130]:
# ranked only
ranked = tinder_score(user_id=u, k=20, pool_k=500, max_km=50)
ranked_ids = ranked["user_id"].tolist()
ranked.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225


,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,38159,0.836038,0.742732,1.0,1.0,1.0,26.175450,"san carlos, california",32,m,straight
1,3717,0.830340,0.734164,1.0,1.0,1.0,31.203141,"palo alto, california",37,m,straight
2,15046,0.827298,0.728997,1.0,1.0,1.0,23.920071,"san francisco, california",36,m,straight
3,16398,0.825377,0.725979,1.0,1.0,1.0,31.203141,"palo alto, california",35,m,straight
4,35672,0.825033,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight
5,10444,0.823391,0.722858,1.0,1.0,1.0,19.197039,"berkeley, california",22,m,straight
6,47397,0.823200,0.722558,1.0,1.0,1.0,23.920071,"san francisco, california",32,m,straight
7,31015,0.823117,0.723199,1.0,1.0,1.0,23.920071,"san francisco, california",31,m,straight
8,41489,0.822003,0.720677,1.0,1.0,1.0,31.716076,"belvedere tiburon, california",23,m,straight
9,17586,0.820568,0.718421,1.0,1.0,1.0,23.920071,"san francisco, california",24,m,straight


In [129]:
# suggestions (no overlap)
suggested = tinder_suggestions(user_id=u, random_sample=20, pool_k=500, max_km=50,
                               weights={"text":0.70,"age":0.10,"life":0.15,"loc":0.05})
suggested.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225


,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,source
0,10833,0.798922,0.699155,1.0,1.00,1.0,30.700929,"el sobrante, california",33,m,straight,suggested
1,54851,0.797583,0.698112,1.0,1.00,1.0,23.920071,"san francisco, california",27,m,straight,suggested
2,48244,0.796747,0.695700,1.0,1.00,1.0,19.197039,"berkeley, california",23,m,straight,suggested
3,11226,0.792767,0.689729,1.0,1.00,1.0,23.920071,"san francisco, california",31,m,straight,suggested
4,53171,0.789856,0.685749,1.0,1.00,1.0,28.113170,"daly city, california",26,m,straight,suggested
5,8379,0.787841,0.682148,1.0,1.00,1.0,16.481152,"emeryville, california",36,m,straight,suggested
6,46695,0.781504,0.726020,1.0,0.75,1.0,31.203141,"palo alto, california",31,m,straight,suggested
7,57727,0.774239,0.661552,1.0,1.00,1.0,29.764458,"san mateo, california",22,m,straight,suggested
8,13060,0.773779,0.661054,1.0,1.00,1.0,23.920071,"san francisco, california",40,m,straight,suggested
9,14484,0.773126,0.659883,1.0,1.00,1.0,23.920071,"san francisco, california",26,m,straight,suggested


In [131]:
tinder_suggestions_df = pd.concat([ranked.head(k).assign(source="ranked"), suggested],
    ignore_index=True).drop_duplicates("user_id", keep="first")

tinder_suggestions_df.head(15)

,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,source
0,38159,0.836038,0.742732,1.0,1.0,1.0,26.175450,"san carlos, california",32,m,straight,ranked
1,3717,0.830340,0.734164,1.0,1.0,1.0,31.203141,"palo alto, california",37,m,straight,ranked
2,15046,0.827298,0.728997,1.0,1.0,1.0,23.920071,"san francisco, california",36,m,straight,ranked
3,16398,0.825377,0.725979,1.0,1.0,1.0,31.203141,"palo alto, california",35,m,straight,ranked
4,35672,0.825033,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight,ranked
5,10444,0.823391,0.722858,1.0,1.0,1.0,19.197039,"berkeley, california",22,m,straight,ranked
6,47397,0.823200,0.722558,1.0,1.0,1.0,23.920071,"san francisco, california",32,m,straight,ranked
7,31015,0.823117,0.723199,1.0,1.0,1.0,23.920071,"san francisco, california",31,m,straight,ranked
8,41489,0.822003,0.720677,1.0,1.0,1.0,31.716076,"belvedere tiburon, california",23,m,straight,ranked
9,17586,0.820568,0.718421,1.0,1.0,1.0,23.920071,"san francisco, california",24,m,straight,ranked


In [142]:
def add_reasons_simple(df_in):
    df = df_in.copy()

    def r(row):
        parts = []
        # text similarity
        if "text_sim" in row and pd.notna(row.text_sim):
            parts.append(f"text~{row.text_sim:.2f}")
        # age rule
        if "age_score" in row and pd.notna(row.age_score):
            parts.append(f"age={int(row.age_score)}")
        # lifestyle
        if "lifestyle_score" in row and pd.notna(row.lifestyle_score):
            parts.append(f"life~{row.lifestyle_score:.2f}")
        # distance
        if "distance_km" in row and pd.notna(row.distance_km):
            parts.append(f"{int(round(row.distance_km))} km")
        # source tag (ranked/suggested)
        if "source" in row and pd.notna(row.source):
            parts.append(str(row.source))
        return " • ".join(parts)

    df["reasons"] = df.apply(r, axis=1)
    return df

In [143]:
tinder_suggestions_df = add_reasons_simple(tinder_suggestions_df)
tinder_suggestions_df[["user_id","tinder_score","reasons"]].head(15)

,user_id,tinder_score,reasons
0,38159,0.836038,text~0.74 • age=1 • life~1.00 • 26 km • ranked
1,3717,0.830340,text~0.73 • age=1 • life~1.00 • 31 km • ranked
2,15046,0.827298,text~0.73 • age=1 • life~1.00 • 24 km • ranked
3,16398,0.825377,text~0.73 • age=1 • life~1.00 • 31 km • ranked
4,35672,0.825033,text~0.73 • age=1 • life~1.00 • 13 km • ranked
5,10444,0.823391,text~0.72 • age=1 • life~1.00 • 19 km • ranked
6,47397,0.823200,text~0.72 • age=1 • life~1.00 • 24 km • ranked
7,31015,0.823117,text~0.72 • age=1 • life~1.00 • 24 km • ranked
8,41489,0.822003,text~0.72 • age=1 • life~1.00 • 32 km • ranked
9,17586,0.820568,text~0.72 • age=1 • life~1.00 • 24 km • ranked


In [136]:
df_rank = ranked.copy()
df_sugg = suggested.copy() 
df_comb = tinder_suggestions_df.copy()

def _brief(df, name):
    print(f"\n=== {name} ({len(df)}) ===")
    cols = [c for c in ["tinder_score","final_score","text_sim","age_score","lifestyle_score","loc_score","distance_km"] if c in df.columns]
    print(df[cols].describe(percentiles=[.05,.25,.5,.75,.95]).round(3))

_brief(df_rank, "RANKED")
_brief(df_sugg, "SUGGESTED")
_brief(df_comb, "COMBINED")


=== RANKED (20) ===
       tinder_score  text_sim  age_score  lifestyle_score  loc_score  \
count        20.000    20.000       20.0             20.0       20.0   
mean          0.821     0.720        1.0              1.0        1.0   
std           0.006     0.009        0.0              0.0        0.0   
min           0.816     0.711        1.0              1.0        1.0   
5%            0.816     0.711        1.0              1.0        1.0   
25%           0.816     0.712        1.0              1.0        1.0   
50%           0.820     0.718        1.0              1.0        1.0   
75%           0.824     0.724        1.0              1.0        1.0   
95%           0.831     0.735        1.0              1.0        1.0   
max           0.836     0.743        1.0              1.0        1.0   

       distance_km  
count       20.000  
mean        24.425  
std          4.390  
min         13.449  
5%          18.910  
25%         23.920  
50%         23.920  
75%         26.066

In [ ]:
"""
Ranked (top-20 by score)
-tinder_score: very tight (0.816 → 0.836), mean ≈ 0.821.
- text_sim: high (mean 0.720), narrow spread.
- age_score, lifestyle_score, loc_score: all flat = 1.0.
- distance: ~24 km average.
For Ranked matches → model found candidates who are already perfect matches on non-text features, 
so text similarity is doing all the heavy lifting.

Suggested (20 random from candidate pool)
- tinder_score: lower (0.724 → 0.799), mean ≈ 0.772.
- text_sim: lower and more variable (mean 0.676, std 0.021).
- lifestyle_score: not always perfect (mean 0.938, std 0.129).
- distance: slightly lower mean (23.3 km), but more spread (7.5 → 31 km).
These are “wildcards”: they’re still viable matches (age/location ok), but they sometimes fail lifestyle 
or text, giving more diversity.

Combined (ranked ∪ suggested, n=30 after dedupe)
The combined set is a blend: solid top matches + a handful of varied suggestions, 
like Tinder mixing "best" with "fresh faces."
""""

In [168]:
# ensure bio_text is present
if "bio_text" not in tinder_suggestions_df.columns:
    tinder_suggestions = tinder_suggestions_df.merge(df[["user_id", "bio_text"]],on="user_id",how="left")

In [163]:
HOBBY_KEYWORDS = {"hiking","hike","climb","gym","fitness","yoga","run","running","bike","cycling","swim","travel","trip",
                  "roadtrip","explore","music","concert","festival","dj","guitar","piano","food","restaurant","cook","cooking",
                  "bake","baking","coffee","brunch","movie","film","cinema","netflix","series","tv","anime",
                  "art","museum","gallery","paint","painting","draw","photography","dog","dogs","cat","cats","pet","pets",
                  "boardgames","games","gaming","reading","books"}

def _highlight_keywords(text, max_hits=6):
    if not isinstance(text, str) or not text:
        return ""
    lower = text.lower()
    hits = sorted({kw for kw in HOBBY_KEYWORDS if kw in lower})
    return " • ".join(hits[:max_hits])

In [166]:
def add_reasons(df_in, bio_col="bio_text"):
    df = df_in.copy()

    def r(row):
        parts = []
        if "text_sim" in row and pd.notna(row.text_sim) and row.text_sim > 0.70:
            parts.append("You write alike")
        if "age_score" in row and pd.notna(row.age_score) and row.age_score == 1:
            parts.append("Similar age")
        if "lifestyle_score" in row and pd.notna(row.lifestyle_score) and row.lifestyle_score > 0.8:
            parts.append("Lifestyle match")
        if "distance_km" in row and pd.notna(row.distance_km) and row.distance_km < 10:
            parts.append("Nearby")
        if "source" in row and pd.notna(row.source):
            parts.append(f"From {row.source}")
        return " • ".join(parts)

    df["reasons"] = df.apply(r, axis=1)
    if bio_col not in df.columns:
        df[bio_col] = ""
    df["details"] = df[bio_col].apply(_highlight_keywords)   # <- hobbies only
    return df


In [169]:
# Safe: no duplicate columns, works whether or not bio_text already exists
bio_map = df.set_index("user_id")["bio_text"]

# If you want to overwrite any existing bio_text with the master copy:
tinder_suggestions_df["bio_text"] = tinder_suggestions_df["user_id"].map(bio_map)

# Now add reasons/hobbies
tinder_suggestions_df = add_reasons(tinder_suggestions_df, bio_col="bio_text")
tinder_suggestions_df[["user_id","tinder_score","reasons","details"]].head(15)

,user_id,tinder_score,reasons,details
0,38159,0.836038,You write alike • Similar age • Lifestyle matc...,art • cook • cooking • gym • reading • travel
1,3717,0.830340,You write alike • Similar age • Lifestyle matc...,art • books • food • music • pet • pets
2,15046,0.827298,You write alike • Similar age • Lifestyle matc...,coffee • concert • fitness • food • music • tv
3,16398,0.825377,You write alike • Similar age • Lifestyle matc...,art • books • cat • coffee • food • guitar
4,35672,0.825033,You write alike • Similar age • Lifestyle matc...,art • books • cat • concert • dog • dogs
5,10444,0.823391,You write alike • Similar age • Lifestyle matc...,art • bike • books • explore • film • food
6,47397,0.823200,You write alike • Similar age • Lifestyle matc...,art • books • cook • cooking • food • music
7,31015,0.823117,You write alike • Similar age • Lifestyle matc...,art • bike • books • cat • cook • cooking
8,41489,0.822003,You write alike • Similar age • Lifestyle matc...,art • cook • dog • dogs • movie • music
9,17586,0.820568,You write alike • Similar age • Lifestyle matc...,art • books • cat • food • games • movie
